<a href="https://colab.research.google.com/github/franciellevargas/MFTCXplain/blob/main/MFTCXplain_GPT4oMini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompting MFTC dataset in Portoguese, English, Italian and Persian with GPT-4o-Mini

#Importing docs

In [ ]:
#he SINGLE most dominant individual moral foundation value expressed in the text, choosing EXACTLY one from this list(lowercase, no quotes, no additional text):care, harm, fairness, cheating, authority, subversion, sanctity, degradation, loyalty, betrayal. If no clear moral value is present, write 'None'T
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

!pip install tqdm openai --quiet
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import os
pd.options.display.max_colwidth = 500
from sklearn.metrics import cohen_kappa_score


# Part 1: GPT-4o-Mini-English

In [ ]:
output_path= ".../MFTCXPLAIN_EMNLP_2025/english_gpt_llama_results"

class gpt_instruct_english:
    def mftc_zero_shot(text, model_name='gpt-4o-mini'):
      prompt= f"""Analyze the following text "{text}" for hate speech and identify its underlying moral value dimensions:
      The Moral Foundations Theory framework represents core ethical ad psychological concerns that come in paired positive vs negative expressions:
      - care vs harm: Involves concern for the well-being of others, with virtues expressed through care, protection, or nurturance, and vices involving harm, cruelty, or indifference to suffering.
      - fairness vs cheating: morals related to justice, rights, and reciprocity, with fairness indicating equity, rule-following, and cheating denoting exploitation, dishonesty, or manipulation.
      - loyalty vs betrayal: morals related to group-based morality, where loyalty refers to solidarity, allegiance, and in-group defense, while betrayal signals disloyalty or abandonment of one’s group.
      - authority vs subversion: morals related to respect for tradition, and legitimate hierarchies, with authority indicating respect or deference to leadership or norms, and subversion indicating rebellion, disrespect, or disobedience.
      - sanctity vs degradation: morals related to purity, contamination, with Purity is associated with cleanliness, modesty, or moral elevation, while degradation includes defilement, obscenity,or perceived corruption.

      Provide your analysis in this exact format:
      hate_label: [YES if the text contains hate speech, NO otherwise]
      moral_value: [the single most prominent moral foundation from: care, harm, fairness, cheating, authority, subversion, sanctity, degradation, loyalty, betrayal. If no clear moral foundation applies, write "None"]
      explanation: [provide a brief evidence based justification, specifically highlighting the words or phrases that triggered your moral value classification. If none, write "None"]

      Provide ONLY the required output format with no additional text, explanations, or justifications."""

      client = OpenAI(
          api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()

      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

    def mftc_few_with_zero_shot(text, model_name='gpt-4o-mini'):
      prompt = f"""your task is to analyze text '{text} for hate speech and identify its underlying moral value dimensions.
      The Moral Foundations Theory framework represents core ethical and psychological concerns that come in paired positive vs negative expressions:
      - care vs harm: Involves concern for the well-being of others, with virtues expressed through care, protection, or nurturance, and vices involving harm, cruelty, or indifference to suffering.
      - fairness vs cheating: morals related to justice, rights, and reciprocity, with fairness indicating equity, rule-following, and cheating denoting exploitation, dishonesty, or manipulation.
      - loyalty vs betrayal: morals related to group-based morality, where loyalty refers to solidarity, allegiance, and in-group defense, while betrayal signals disloyalty or abandonment of one’s group.
      - authority vs subversion: morals related to respect for tradition, and legitimate hierarchies, with authority indicating respect or deference to leadership or norms, and subversion indicating rebellion, disrespect, or disobedience.
      - sanctity vs degradation: morals related to purity, contamination, with Purity is associated with cleanliness, modesty, or moral elevation, while degradation includes defilement, obscenity,or perceived corruption.

      Examples:
          text: @USER @USER Leftists showing yet again how vile they can be
          hate_label: YES
          moral_value: degradation
          explanation: vile

          text: @USER if the reason for Farakans illegal values are because he is black or Muslim, that would be a reason to detest him. I believe his illegal values are simply a result of bad character, like his supporters!
          hate_label: NO
          moral_value: fairness
          explanation: if the reason for Farakans illegal values are because he is black or Muslim, that would be a reason to detest him. I believe his illegal values are simply a result of bad character,.

          text: @USER @USER @USER Bullshit! Regulations are good because it keeps conservatives from striping our land into the Abyss
          hate_label: NO
          moral_value: betrayal
          explanation: it keeps conservatives from striping our land into the Abyss

          text: @USER @USER @USER @USER Only a liberal would support a liberal that spent a MILLION to get liberals elected to office. #LibFAIL! URL
          hate_label: YES
          moral_value: subversion
          explanation: to get liberals elected to office

      Respond using the same format as the examples above, with no additional text, explanations, or justifications.
      hate_label: [YES if the text contains hate speech, NO otherwise]
      moral_value: [the single most dominant moral value from: care, harm, fairness, cheating, authority, subversion, sanctity, degradation, loyalty, betrayal. If no clear moral value applies, write "None"]
      explanation: [a brief evidence based justification, specifically highlighting the words or phrases that triggered your moral value classification. If none, write "None"]
      """

      client = OpenAI(
          api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()

      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

    def mftc_cot(text, model_name='gpt-4o-mini'):
      prompt = f"""your task is to analyze text '{text} for hate speech and identify its underlying moral value dimensions.
      Follow these steps:
      step 1: Determine if the text contains hate speech or not. YES if the text contains hate speech, NO otherwise.
      step 2: The Moral Foundations Theory framework represents core ethical and psychological concerns that come in paired positive vs negative expressions:
      - care vs harm: Involves concern for the well-being of others, with virtues expressed through care, protection, or nurturance, and vices involving harm, cruelty, or indifference to suffering.
      - fairness vs cheating: morals related to justice, rights, and reciprocity, with fairness indicating equity, rule-following, and cheating denoting exploitation, dishonesty, or manipulation.
      - loyalty vs betrayal: morals related to group-based morality, where loyalty refers to solidarity, allegiance, and in-group defense, while betrayal signals disloyalty or abandonment of one’s group.
      - authority vs subversion: morals related to respect for tradition, and legitimate hierarchies, with authority indicating respect or deference to leadership or norms, and subversion indicating rebellion, disrespect, or disobedience.
      - sanctity vs degradation: morals related to purity, contamination, with Purity is associated with cleanliness, modesty, or moral elevation, while degradation includes defilement, obscenity,or perceived corruption.

      step 3: select the most dominant moral foundation as described in step 2 , choosing EXACTLY one from this list: harm, fairness, cheating, authority, subversion, sanctity, degradation, loyalty, betrayal. If no clear moral value applies, write "None".
      step 4: provide a brief evidence based justification for your choice in step 3, specifically highlighting the word(s) or phrases that triggered your moral foundation classification.
      Format your response as:
      hate_label: [YES or NO]
      moral_value: [single moral foundation chosen from step 3. If no clear moral value applies, write "None"]
      explanation: [brief justification from step 4. If none, write "None"]

      Provide ONLY the required output format with no additional text, explanations, or justifications."""

      client = OpenAI(
          api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()

      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

dataset_english = pd.read_csv('.../MFTCXPLAIN_EMNLP_2025/MFTCxplain-english - english_final.csv')
dataset_english.set_index('id', inplace=True)
print(f"The shape of the entire dataset: {dataset_english.shape}")
#removing these id because they were used as 4 shot examples
rows_to_remove = [9, 17, 19, 29]

testing = dataset_english.drop(index=rows_to_remove)
print(f"\nThe shape of the entire dataset minus 4 rows:{testing.shape}")

confirming = dataset_english.iloc[[9, 17, 19, 29]]
print('\nJust confirming the examples from the few show we will be excluding:\n ')
confirming

The shape of the entire dataset: (1050, 15)

The shape of the entire dataset minus 4 rows:(1046, 15)

Just confirming the examples from the few show we will be excluding:
 


,hate_speech,texts,label1_annotator1,rationales_1,label2_annotator1,rationales_2,label3_annotator1,rationales_3,label1_annotator2,rationales_1.1,label2_annotator2,rationales_2.1,label3_annotator2,rationales_3.1,platform
id,,,,,,,,,,,,,,,
10,0,"“Fiquei sabendo pelo diário oficial, não assinei...” Moro, Sergio. Alô, @MPF_PGR, alguém publicou documento falso no DOU, hein?!",FN - Injustiça,"Fiquei sabendo pelo diário oficial, não assinei...”",NaN,NaN,NaN,NaN,FN - Injustiça,não assinei...; publicou documento falso,NaN,NaN,NaN,NaN,twitter
18,0,"“O presidente [Jair Bolsonaro] não me quer no cargo”, afirmou o ex-juiz da Lava Jato. Moro diz que presidente rompeu promessa de lhe dar carta branca na pasta e no combate à corrupção. https://t.co/CzWI2t7J3t",LN - Traição,Moro diz que presidente rompeu promessa de lhe dar carta branca na pasta e no combate à corrupção,FN - Injustiça,O presidente [Jair Bolsonaro] não me quer no cargo;,NaN,NaN,LN - Traição,não me quer no cargo; presidente rompeu promessa,NaN,NaN,NaN,NaN,twitter
20,0,“O Presidente não me quer no cargo” _ Sérgio Moro,FN - Injustiça,O Presidente não me quer no cargo;,NaN,NaN,NaN,NaN,LN - Traição,O Presidente não me quer no cargo;,NaN,NaN,NaN,NaN,twitter
30,0,"“Vou procurar, mais adiante, um emprego. Não enriqueci.” - Disse o ex-homem forte do governo Jair Bolsonaro Sergio Moro",AP - Autoridade,Disse o ex-homem forte do governo Jair Bolsonaro Sergio Moro,NaN,NaN,NaN,NaN,AP - Autoridade,Disse o ex-homem forte do governo Jair Bolsonaro Sergio Moro,NaN,NaN,NaN,NaN,twitter


In [ ]:
gpt_prompts_english = gpt_instruct_english()

# 4 Shot prompting strategy
explain_few = dataset_english.drop(index=rows_to_remove)
explain_few[['hate_label', 'moral_value', 'explanation']] = explain_few['texts'].progress_apply(lambda x: pd.Series(gpt_prompts_english.mftc_few_with_zero_shot(x)))
file_name = f'gpt_english_mftc_explain_4_shot_style_prompt.csv'
file_path = os.path.join(output_path , file_name)
explain_few.to_csv(file_path, index=False)

# Zero Shot
explain_zero = dataset_english.drop(index=rows_to_remove)
explain_zero[['hate_label', 'moral_value', 'explanation']] = explain_zero['texts'].progress_apply(lambda x: pd.Series(gpt_prompts_english.mftc_zero_shot(x)))
file_name = f'gpt_english_mftc_explain_zero.csv'
file_path = os.path.join(output_path , file_name)
explain_zero.to_csv(file_path, index=False)

#Chain of Thought
llama_prompts = llama_instruct
explain_cot = dataset_english.drop(index=rows_to_remove)
explain_cot[['hate_label', 'moral_value', 'explanation']] = explain_cot['tweets'].progress_apply(lambda x: pd.Series(gpt_prompts_english.mftc_cot(x)))
file_name = f'gpt_english_explain_cot.csv'
file_path = os.path.join(output_path , file_name)
explain_cot.to_csv(file_path, index=False)

# Part 2: GPT Itali

In [ ]:
output_path= ".../MFTCXPLAIN_EMNLP_2025/itali_gpt_llama_results"

class gpt_instruct_itali:
    def mftc_zero_shot(text, model_name='gpt-4o-mini'):
      prompt= f"""Analizza il seguente testo "{text}" per identificare discorsi d'odio e individuare i relativi valori morali presenti:
      La teoria dei fondamenti morali ("Moral Foundation Theory") descrive i principali valori etici e psicologici che possono manifestarsi in formulazioni positive e negative:
      - cura vs danno: Riguarda l'attenzione per il benessere altrui, con virtù espresse attraverso assistenza, protezione o sostegno, e vizi che comportano danno, crudeltà o indifferenza alla sofferenza.
      - equità vs imbroglio: Valori morali legati a giustizia, diritti e reciprocità, dove l'equità rappresenta parità, rispetto delle regole, mentre l'imbroglio indica sfruttamento, disonestà o manipolazione.
      - lealtà vs tradimento: Valori morali relativi all'etica di gruppo, dove la lealtà si riferisce a solidarietà, fedeltà e difesa del proprio gruppo, mentre il tradimento indica slealtà o abbandono del proprio gruppo.
      - autorità vs sovversione: Valori morali concernenti il rispetto per la tradizione e le gerarchie legittime, dove l'autorità rappresenta rispetto o deferenza verso la leadership o le norme stabilite, mentre la sovversione indica ribellione, mancanza di rispetto o disobbedienza.
      - santità vs degradazione: Valori morali relativi a purezza e contaminazione, dove la santità è associata a pulizia, modestia o elevazione morale, mentre la degradazione comprende profanazione, oscenità o percepita corruzione.

      Presenta la tua analisi seguendo esattamente questo formato:
      hate_label: [YES se il testo contiene discorsi d'odio, NO altrimenti]
      moral_value: [il valore morale più presente nel testo tra: cura, danno, equità, imbroglio, autorità, sovversione, santità, degradazione, lealtà, tradimento. Se non si applica alcun valore morale, scrivi "None"]
      explanation: [fornisci una breve giustificazione basata sul testo per la tua scelta, riportando esattemente le parole o frasi che hanno indotto la tua classificazione del valore morale Se nessuna, scrivi "None"]

      Rispondi seguendo SOLO il formato richiesto senza aggiungere testo, spiegazioni o giustificazioni."""

      client = OpenAI(
      api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()

      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

    def mftc_few_with_zero_shot(text, model_name='gpt-4o-mini'):
      prompt = f"""il tuo compito è quello di analizzare il testo '{text}' per individuare discorsi d'odio e relativi valori morali presenti.
      La teoria dei fondamenti morali ("Moral Foundation Theory") descrive i principali valori etici e psicologici che possono manifestarsi in formulazioni positive e negative:
      - cura vs danno: Riguarda l'attenzione per il benessere altrui, con virtù espresse attraverso assistenza, protezione o sostegno, e vizi che comportano danno, crudeltà o indifferenza alla sofferenza.
      - equità vs imbroglio: Valori morali legati a giustizia, diritti e reciprocità, dove l'equità rappresenta parità, rispetto delle regole, mentre l'imbroglio indica sfruttamento, disonestà o manipolazione.
      - lealtà vs tradimento: Valori morali relativi all'etica di gruppo, dove la lealtà si riferisce a solidarietà, fedeltà e difesa del proprio gruppo, mentre il tradimento indica slealtà o abbandono del proprio gruppo.
      - autorità vs sovversione: Valori morali concernenti il rispetto per la tradizione e le gerarchie legittime, dove l'autorità rappresenta rispetto o deferenza verso la leadership o le norme stabilite, mentre la sovversione indica ribellione, mancanza di rispetto o disobbedienza.
      - santità vs degradazione: Valori morali relativi a purezza e contaminazione, dove la santità è associata a pulizia, modestia o elevazione morale, mentre la degradazione comprende profanazione, oscenità o percepita corruzione.

      Esempi:
      testo: Cédric Herrou, contadino francese che ha ospitato migliaia di migranti di passaggio lungo il confine nei pressi di Ventimiglia, è stato assolto dal Consiglio Costituzionale transalpino. Motivazione? La “fraternità” è pilastro del diritto della Francia repubblicana. #RestiamoUmani
      hate_label: NO
      moral_value: equità
      explanation: 'pilastro del diritto'.

      testo: Manifestano contro il #greenpass,protestano i #novax,assaltano la #Cgil con i fascisti di Forza Nuova, un tutore dell'ordine incita la folla a disobbedire. E pensare che paesi poveri del terzo mondo sognano il vaccino.Noi abbiamo un vaccino sicuro e gratis e i cretini in piazza.
      hate_label: YES
      moral_value: sovversione
      explanation: 'Manifestano contro il #greenpass,protestano i #novax,assaltano la #Cgil con i fascisti di Forza Nuova, un tutore dell'ordine incita la folla a disobbedire'

      testo: PRIMA LE MASCHERINE. ... ORA I VACCINI. COSA FATE ANCORA A RUBARCI SOLDI X PAGARVI VOSTRI STIPENDI? APRITEEEEEEEEEEE VERGIGNOSI TUTTI CON LE TASCHE PIENE....#Drittoerovescio
      hate_label: YES
      moral_value: imbroglio
      explanation: 'COSA FATE ANCORA A RUBARCI SOLDI X PAGARVI VOSTRI STIPENDI'

      testo: L’Italia è leghista, non più cristiana. Il leghista dice: “prima gli italiani “; il Cristiano: “prima gli scartati”. Né basta baciare in pubblico Gesù; l’ha già fatto anche Giuda.
      hate_label: NO
      moral_value: lealtà
      explanation: Il leghista dice: "prima gli italiani"

      Rispondi utilizzando lo stesso formato degli esempi sopra, senza aggiungere nessun testo, spiegazioni o giustificazioni.
      hate_label: [YES se il testo contiene discorsi d'odio, NO altrimenti]
      moral_value: [il valore morale singolo più presente nel testo tra: cura, danno, equità, imbroglio, autorità, sovversione, santità, degradazione, lealtà, tradimento. Se non c’e’ alcun valore morale, scrivi "None"]
      explanation: [fornisci una breve giustificazione basata sul testo per la tua scelta, riportando esattemente le parole o frasi che hanno indotto la tua classificazione del valore morale. Se nessuna, scrivi "None"]"""

      client = OpenAI(
      api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()

      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

    def mftc_cot(text, model_name='gpt-4o-mini'):
      prompt = f"""il tuo compito è quello di analizzare il testo '{text}' per individuare discorsi d'odio e relativi valori morali presenti.
      Procedi seguendo questi step:
      Step 1: Determina se il testo contiene discorsi d'odio oppure no. YES se il testo contiene discorsi d'odio, NO altrimenti.
      Step 2: La teoria dei fondamenti morali ("Moral Foundation Theory") descrive i principali valori etici e psicologici che possono manifestarsi in formulazioni positive e negative:
      - cura vs danno: Riguarda l'attenzione per il benessere altrui, con virtù espresse attraverso assistenza, protezione o sostegno, e vizi che comportano danno, crudeltà o indifferenza alla sofferenza.
      - equità vs imbroglio: Valori morali legati a giustizia, diritti e reciprocità, dove l'equità rappresenta parità, rispetto delle regole, mentre l'imbroglio indica sfruttamento, disonestà o manipolazione.
      - lealtà vs tradimento: Valori morali relativi all'etica di gruppo, dove la lealtà si riferisce a solidarietà, fedeltà e difesa del proprio gruppo, mentre il tradimento indica slealtà o abbandono del proprio gruppo.
      - autorità vs sovversione: Valori morali concernenti il rispetto per la tradizione e le gerarchie legittime, dove l'autorità rappresenta rispetto o deferenza verso la leadership o le norme stabilite, mentre la sovversione indica ribellione, mancanza di rispetto o disobbedienza.
      - santità vs degradazione: Valori morali relativi a purezza e contaminazione, dove la santità è associata a pulizia, modestia o elevazione morale, mentre la degradazione comprende profanazione, oscenità o percepita corruzione.

      Step 3: seleziona il valore morale più presente nel testo come descritto nello Step 2, scegliendo SOLTANTO una da questo elenco: danno, equità, imbroglio, autorità, sovversione, santità, degradazione, lealtà, tradimento. Se non si applica alcun chiaro valore morale, scrivi "Nessuno".
      Step 4: fornisci una breve giustificazione basata sul testo per la tua scelta nello Step 3, riportando esattemente le parole o frasi che hanno indotto la tua classificazione del valore morale.
      Riporta la tua risposta come segue:
      hate_label: [YES o NO]
      moral_value: [il valore morale scelto nello Step 3. Se non c’e’ alcun valore morale, scrivi "None"]
      explanation: [breve giustificazione dello Step 4. Se nessuna, scrivi "None"]

      Rispondi seguendo SOLO il formato richiesto senza aggiungere testo, spiegazioni o giustificazioni"""

      client = OpenAI(
      api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()

      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

dataset_itali = pd.read_excel('.../MFTCXPLAIN_EMNLP_2025/MFTCXplain-italian.xlsx')

dataset_itali.set_index('id', inplace=True) #reindex to the original 'id' index
print(f"The shape of the entire dataset: {dataset_itali.shape}")
#removing these id because they were used as 4 shot examples
rows_to_remove = [9, 17, 19, 29]
testing = dataset_itali.drop(index=rows_to_remove)
print(f"\nThe shape of the entire dataset minus 4 rows:{testing.shape}")

confirming = dataset_itali.iloc[[9, 17, 19, 29]]
print('\nJust confirming the examples from the few show we will be excluding:\n ')
confirming

Mounted at /content/drive
The shape of the entire dataset: (621, 9)

The shape of the entire dataset minus 4 rows:(617, 9)

Just confirming the examples from the few show we will be excluding:
 


,texts,hate_speech,mft_sentiment_first,rationales_first,mft_sentiment_second,rationales_second,mft_sentiment_third,rationales_third,platform
id,,,,,,,,,
10,"Cédric Herrou, contadino francese che ha ospitato migliaia di migranti di passaggio lungo il confine nei pressi di Ventimiglia, è stato assolto dal Consiglio Costituzionale transalpino.\nMotivazione? La “fraternità” è pilastro del diritto della Francia repubblicana.\n#RestiamoUmani\n",0,FP - Fairness,pilastro del diritto,HP - Care,ha ospitato migliaia di migranti,LP - Loyaty,La “fraternità” è pilastro del diritto della Francia repubblicana.,Twitter
18,"Manifestano contro il #greenpass,protestano i #novax,assaltano la #Cgil con i fascisti di Forza Nuova, un tutore dell'ordine incita la folla a disobbedire. E pensare che paesi poveri del terzo mondo sognano il vaccino.Noi abbiamo un vaccino sicuro e gratis e i cretini in piazza.",1,AN - Subversion,"Manifestano contro il #greenpass,protestano i #novax,assaltano la #Cgil con i fascisti di Forza Nuova, un tutore dell'ordine incita la folla a disobbedire",NaN,NaN,NaN,NaN,Twitter
20,PRIMA LE MASCHERINE. ... ORA I VACCINI. COSA FATE ANCORA A RUBARCI SOLDI X PAGARVI VOSTRI STIPENDI? APRITEEEEEEEEEEE VERGIGNOSI TUTTI CON LE TASCHE PIENE....#Drittoerovescio,1,FN - Cheating,COSA FATE ANCORA A RUBARCI SOLDI X PAGARVI VOSTRI STIPENDI,NaN,NaN,NaN,NaN,Twitter
30,"L’Italia è leghista, non più cristiana. Il leghista dice: “prima gli italiani “; il Cristiano: “prima gli scartati”. Né basta baciare in pubblico Gesù; l’ha già fatto anche Giuda.",0,LP - Loyaty,"Il leghista dice: ""prima gli italiani""",PP - Purity,"il Cristiano dice: ""prima gli scartati""",NaN,NaN,Twitter


In [ ]:
gpt_prompts_itali = gpt_instruct_itali()
output_path = '.../MFTCXPLAIN_EMNLP_2025/italia_gpt_llama_results'

#4-shot
explain_few = dataset_itali.drop(index=rows_to_remove)
explain_few[['hate_label', 'moral_value', 'explanation']] = explain_few['texts'].progress_apply(lambda x: pd.Series(gpt_prompts_itali.mftc_few_with_zero_shot(x)))
file_name = f'gpt_itali_mftc_explain_4_shot_with_zero_shot_style_prompt.csv'
file_path = os.path.join(output_path , file_name)
explain_few.to_csv(file_path, index=False)

# Zero Shot
explain_zero = dataset_itali.drop(index=rows_to_remove)
explain_zero[['hate_label', 'moral_value', 'explanation']] = explain_zero['texts'].progress_apply(lambda x: pd.Series(gpt_prompts_itali.mftc_zero_shot(x)))
file_name = f'gpt_itali_mftc_explain_zero.csv'
file_path = os.path.join(output_path , file_name)
explain_zero.to_csv(file_path, index=False)

#Chain of Thought
explain_cot = dataset_itali.drop(index=rows_to_remove)
explain_cot[['hate_label', 'moral_value', 'explanation']] = explain_cot['texts'].progress_apply(lambda x: pd.Series(gpt_prompts_itali.mftc_cot(x)))
file_name = f'gpt_itali_mftc_explain_cot.csv'
file_path = os.path.join(output_path , file_name)
explain_cot.to_csv(file_path, index=False)

#Part 3: GPT Portugal

In [ ]:
output_path= ".../MFTCXPLAIN_EMNLP_2025/porto_gpt_llama_results"
class gpt_instruct_porto:
    def mftc_zero_shot(text, model_name='gpt-4o-mini'):
      prompt= f"""Analise o seguinte texto "{text}" para discurso de ódio e identificação de suas dimensões morais subjacentes:
      A Teoria dos Fundamentos Morais representa preocupações éticas e psicológicas centrais que aparecem em pares de categorias positivas versus negativas:
      - cuidado versus dano: envolve preocupação com o bem-estar dos outros, com virtudes expressas por meio de cuidado, proteção ou nutrição, e vícios envolvendo dano, crueldade ou indiferença ao sofrimento do outro.
      - justiça versus trapaça: valores morais relacionados à justiça, direitos e reciprocidade; justiça indica equidade e respeito às regras, enquanto trapaça denota exploração, desonestidade ou manipulação.
      - lealdade versus traição: valores morais ligados à moralidade baseada em grupos, onde lealdade refere-se à solidariedade, fidelidade e defesa do grupo, enquanto traição sinaliza deslealdade ou abandono do próprio grupo.
      - autoridade versus subversão: valores morais relacionados ao respeito pela tradição e hierarquias legítimas; autoridade indica respeito ou deferência à liderança ou normas, e subversão indica rebeldia, desrespeito ou desobediência.
      - santidade versus degradação: valores morais ligados à pureza e contaminação; santidade está associada à limpeza, modéstia ou elevação moral, enquanto degradação inclui profanação ou obscenidade percebida.

      Forneça sua análise exatamente neste formato:
      hate_label: [SIM se o texto apresentar discurso de ódio, NÃO caso o texto não apresente discurso de ódio]
      moral_value: [o fundamento moral mais proeminente entre: cuidado, dano, justiça, trapaça, autoridade, subversão, santidade, degradação, lealdade, traição. Se nenhum fundamento moral claro se aplicar, escreva "Nenhum"]
      explanation: [forneça uma justificativa breve baseada em evidências, destacando especificamente as palavras ou frases que motivaram a classificação do valor moral. Se nenhuma evidencia para classificação de moralidade for encontrada, escreva "Nenhuma"]

      Forneça SOMENTE o formato de saída exigido, sem texto adicional, explicações ou justificações."""

      client = OpenAI(
      api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()
      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

    def mftc_few_with_zero_shot(text, model_name='gpt-4o-mini'):
      prompt = f"""Analise o seguinte texto "{text}" para discurso de ódio e identificação de suas dimensões morais subjacentes:
      A Teoria dos Fundamentos Morais (MFT) representa preocupações éticas e psicológicas centrais que se manifestam em pares de categorias positivas versus negativas:
      - cuidado versus dano: envolve preocupação com o bem-estar dos outros, com virtudes expressas por meio de cuidado, proteção ou nutrição, e vícios envolvendo dano, crueldade ou indiferença ao sofrimento do outro.
      - justiça versus trapaça: valores morais relacionados à justiça, direitos e reciprocidade; justiça indica equidade e respeito às regras, enquanto trapaça denota exploração, desonestidade ou manipulação.
      - lealdade versus traição: valores morais ligados à moralidade baseada em grupos, onde lealdade refere-se à solidariedade, fidelidade e defesa do grupo, enquanto traição sinaliza deslealdade ou abandono do próprio grupo.
      - autoridade versus subversão: valores morais relacionados ao respeito pela tradição e hierarquias legítimas; autoridade indica respeito ou deferência à liderança ou normas, e subversão indica rebeldia, desrespeito ou desobediência.
      - santidade versus degradação: valores morais ligados à pureza e contaminação; santidade está associada à limpeza, modéstia ou elevação moral, enquanto degradação inclui profanação ou obscenidade percebida.

      Exemplos:
      text: 45 minutos de pronunciamento vai tomar no cu Bolsonaro. Parece um episódio da Netflix
      hate_label: SIM
      moral_value: degradação
      explanation: vai tomar no cu Bolsonaro.

      text: Dilma nunca interferiu em nenhuma investigação da lavajato (Sérgio Moro, 2020)
      hate_label: NÃO
      moral_value: justiça
      explanation: nunca interferiu em nenhuma investigação

      text: Tenho muita gratidão e admiração pelo Juiz Sérgio Moro. Respeitoo, muito. Agora, votei em Bolsonaro e nunca esperei que Sérgio Moro virasse Ministro. Continuo fiel a Bolsonaro até que me provem que ele é desonesto. Não entregarei meu pais para a esquerda, sem lutar. Morrer pelo BR
      hate_label: NÃO
      moral_value: lealdade
      explanation: Tenho muita gratidão e admiração pelo Juiz Sérgio Moro. Respeitoo, muito; Continuo fiel a Bolsonaro até que me provem que ele é desonesto; Não entregarei meu pais para a esquerda, sem lutar;  Morrer pelo BR;

      text: @JanainaDoBrasil Basta uma pesquisa de popularidade e outra, o que é que nós estamos esperando para fazer uma carreata de #ForaBolsonaroUrgente ? Tô ansiosa pra ir a uma de máscara, com cartazes pedindo CtrlZ no Mandetta e Sergio Moro e Delete no Bozoró!
      hate_label: SIM
      moral_value: subversão
      explanation: o que é que nós estamos esperando para fazer uma carreata de #ForaBolsonaroUrgente; Tô ansiosa pra ir a uma de máscara, com cartazes pedindo CtrlZ no Mandetta e Sergio Moro e Delete no Bozoró!;

      Responda utilizando o mesmo formato dos exemplos acima, sem texto adicional, explicações ou justificativas.
      hate_label: [SIM se o texto apresentar discurso de ódio, NÃO caso o texto não apresentar discurso de ódio]
      moral_value: [o valor moral mais predominante entre: cuidado, dano, justiça, trapaça, autoridade, subversão, santidade, degradação, lealdade, traição. Se nenhum valor moral claro se aplicar, escreva "Nenhum"]
      explanation: [uma justificativa breve baseada em evidências, destacando especificamente as palavras ou frases que motivaram a classificação do valor moral. Se nenhuma, escreva "Nenhuma"]  """

      client = OpenAI(
      api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()
        # print(hate_label, moral_value, explanation)
      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

    def mftc_cot(text, model_name='gpt-4o-mini'):
      prompt = f"""sua tarefa é analisar o texto "{text}" para classificação de discurso de ódio e identificação de suas dimensões morais subjacentes.
      Siga os seguintes passos:
      passo 1: Determine se o texto contém discurso de ódio ou não. SIM se o texto contiver discurso de ódio, NÃO caso contrário.
      passo 2: A estrutura da Teoria dos Fundamentos Morais (MFT) representa preocupações éticas e psicológicas centrais que aparecem em pares de categorias positivas versus negativas:
      - cuidado versus dano: envolve preocupação com o bem-estar dos outros, com virtudes expressas por meio de cuidado, proteção ou nutrição, e vícios envolvendo dano, crueldade ou indiferença ao sofrimento do outro.
      - justiça versus trapaça: valores morais relacionados à justiça, direitos e reciprocidade; justiça indica equidade e respeito às regras, enquanto trapaça denota exploração, desonestidade ou manipulação.
      - lealdade versus traição: valores morais ligados à moralidade baseada em grupos, onde lealdade refere-se à solidariedade, fidelidade e defesa do grupo, enquanto traição sinaliza deslealdade ou abandono do próprio grupo.
      - autoridade versus subversão: valores morais relacionados ao respeito pela tradição e hierarquias legítimas; autoridade indica respeito ou deferência à liderança ou normas, e subversão indica rebeldia, desrespeito ou desobediência.
      - santidade versus degradação: valores morais ligados à pureza e contaminação; santidade está associada à limpeza, modéstia ou elevação moral, enquanto degradação inclui profanação ou obscenidade percebida.
      passo 3: Selecione o fundamento moral mais dominante, conforme descrito no passo 2, escolhendo EXATAMENTE uma das seguintes categorias: dano, justiça, trapaça, autoridade, subversão, santidade, degradação, lealdade, traição. Se nenhum valor moral claro se aplicar, escreva "Nenhum".
      passo 4: Forneça uma justificativa breve baseada em evidências para a sua escolha no passo 3, destacando especificamente a(s) palavra(s) ou frase(s) que motivaram sua classificação do fundamento moral.
      Formate sua resposta como:
      hate_label: [SIM ou NÃO]
      moral_value: [fundamento moral escolhido no passo 3. Se nenhum valor moral claro se aplicar, escreva "Nenhum"]
      explanation: [justificativa breve do passo 4. Se nenhuma, escreva "Nenhuma"]

      Forneça SOMENTE o formato de saída exigido, sem texto adicional, explicações ou justificativas."""

      client = OpenAI(
      api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()
        # print(hate_label, moral_value, explanation)
      if not hate_label or not moral_value or not explanation:
        return None
      else:
        # print(hate_label, moral_value, explanation)
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }


dataset_porto = pd.read_csv('.../MFTCXPLAIN_EMNLP_2025/MFTCXplain-portuguese - full.csv')

dataset_porto.set_index('id', inplace=True) #reindex to the original 'id' index
print(f"The shape of the entire dataset: {dataset_porto.shape}")
#removing these id because they were used as 4 shot examples
rows_to_remove = [730, 6, 849, 377]
testing = dataset_porto.drop(index=rows_to_remove)
print(f"\nThe shape of the entire dataset minus 4 rows:{testing.shape}")

confirming = dataset_porto.iloc[[730, 6, 849, 377]]
print('\nJust confirming the examples from the few show we will be excluding:\n ')
confirming

The shape of the entire dataset: (1050, 15)

The shape of the entire dataset minus 4 rows:(1046, 15)

Just confirming the examples from the few show we will be excluding:
 


,hate_speech,texts,label1_annotator1,rationales_1,label2_annotator1,rationales_2,label3_annotator1,rationales_3,label1_annotator2,rationales_1.1,label2_annotator2,rationales_2.1,label3_annotator2,rationales_3.1,platform
id,,,,,,,,,,,,,,,
731,1,45 minutos de pronunciamento vai tomar no cu Bolsonaro. Parece um episódio da Netflix,PN - Depravação,vai tomar no cu Bolsonaro;,NaN,NaN,NaN,NaN,PN - Depravação,vai tomar no cu,NaN,NaN,NaN,NaN,twitter
7,0,"“Dilma nunca interferiu em nenhuma investigação da lavajato” (Sérgio Moro, 2020).",FP - Justiça,nunca interferiu em nenhuma investigação,NaN,NaN,NaN,NaN,FP - Justiça,nunca interferiu em nenhuma investigação,NaN,NaN,NaN,NaN,twitter
850,0,"Tenho muita gratidão e admiração pelo Juiz Sérgio Moro. Respeitoo, muito. Agora, votei em Bolsonaro e nunca esperei que Sérgio Moro virasse Ministro. Continuo fiel a Bolsonaro até que me provem que ele é desonesto. Não entregarei meu pais para a esquerda, sem lutar. Morrer pelo BR",LP - Lealdade,"Tenho muita gratidão e admiração pelo Juiz Sérgio Moro. Respeitoo, muito; Continuo fiel a Bolsonaro até que me provem que ele é desonesto; Não entregarei meu pais para a esquerda, sem lutar; Morrer pelo BR;",NaN,NaN,NaN,NaN,LP - Lealdade,votei em Bolsonaro e nunca esperei que Sérgio Moro virasse Ministro; Continuo fiel a Bolsonaro até que me provem que ele é desonesto,HN - Prejudicial,"Não entregarei meu pais para a esquerda, sem lutar; Morrer pelo BR",NaN,NaN,twitter
378,1,"@JanainaDoBrasil Basta uma pesquisa de popularidade e outra, o que é que nós estamos esperando para fazer uma carreata de #ForaBolsonaroUrgente ? Tô ansiosa pra ir a uma de máscara, com cartazes pedindo CtrlZ no Mandetta e Sergio Moro e Delete no Bozoró!",AN - Subversão,"o que é que nós estamos esperando para fazer uma carreata de #ForaBolsonaroUrgente; Tô ansiosa pra ir a uma de máscara, com cartazes pedindo CtrlZ no Mandetta e Sergio Moro e Delete no Bozoró!;",NaN,NaN,NaN,NaN,AN - Subversão,o que é que nós estamos esperando para fazer uma carreata de #ForaBolsonaroUrgente ?,NaN,NaN,NaN,NaN,twitter


In [ ]:
gpt_prompts_porto = gpt_instruct_porto()
# few shot
explain_few = dataset_porto.drop(index=rows_to_remove)
explain_few[['hate_label', 'moral_value', 'explanation']] = explain_few['texts'].progress_apply(lambda x: pd.Series(gpt_prompts_porto.mftc_few_with_zero_shot(x)))
file_name = f'new_gpt_porto_mftc_explain_4_shot_with_zero_shot_style_prompt.csv'
file_path = os.path.join(output_path , file_name)
explain_few.to_csv(file_path, index=False)

#zero
explain_zero = dataset_porto.drop(index=rows_to_remove)
explain_zero[['hate_label', 'moral_value', 'explanation']] = explain_zero['texts'].progress_apply(lambda x: pd.Series(gpt_prompts_porto.mftc_zero_shot(x)))
file_name = f'new_gpt_porto_mftc_explain_zero.csv'
file_path = os.path.join(output_path , file_name)
explain_zero.to_csv(file_path, index=False)

#cot
explain_cot = dataset_porto.drop(index=rows_to_remove)
explain_cot[['hate_label', 'moral_value', 'explanation']] = explain_cot['texts'].progress_apply(lambda x: pd.Series(gpt_prompts_porto.mftc_cot(x)))
file_name = f'new_gpt_porto_mftc_explain_cot.csv'
file_path = os.path.join(output_path , file_name)
explain_cot.to_csv(file_path, index=False)

In [ ]:
explain_cot

,hate_speech,texts,label1_annotator1,rationales_1,label2_annotator1,rationales_2,label3_annotator1,rationales_3,label1_annotator2,rationales_1.1,label2_annotator2,rationales_2.1,label3_annotator2,rationales_3.1,platform,hate_label,moral_value,explanation
id,,,,,,,,,,,,,,,,,,
1034,0,Resumo do pronunciamento do Bolsonaro sobre a demissão de Moro: “Precisamos falar de responsabilidade afetiva” kkkkkkk,HP - Cuidado,Precisamos falar de responsabilidade afetiva”,NaN,NaN,NaN,NaN,HP - Cuidado,Precisamos falar de responsabilidade afetiva”,NaN,NaN,NaN,NaN,NaN,NÃO,Nenhum,Nenhuma
522,1,@OlavoOpressor Sérgio Moro é herói nacional e vc é traidor e corrupto igual ao PT.,FN - Injustiça,corrupto igual ao PT.,LN - Traição,vc é traidor,LP - Lealdade,Sérgio Moro é herói nacional;,FN - Injustiça,corrupto igual ao PT;,LN - Traição,vc é traidor,NaN,NaN,twitter,SIM,traição,"O texto acusa a pessoa mencionada de ser ""traidor"" e ""corrupto"", o que se alinha com a dimensão moral de traição, expressando deslealdade em relação a um grupo ou ideologia."


# Part 4: GPT persia

In [ ]:
output_path= ".../MFTCXPLAIN_EMNLP_2025/persia_gpt_llama_results"
class gpt_instruct_farsi:
    def mftc_zero_shot(text, model_name='gpt-4o-mini'):
      prompt = f"""متن "{text}" زیر را برای گفتار نفرت‌انگیز تحلیل کرده و ابعاد ارزش‌های اخلاقی زیربنایی آن را شناسایی کنید:
      چارچوب نظریه بنیان‌های اخلاقی، دغدغه های اصلی اخلاقی و روان‌شناختی را نشان می‌دهد که به‌صورت جفت‌های مثبت و منفی ظاهر می‌شوند:
      مراقبت در برابر آسیب: شامل نگرانی برای رفاه دیگران است؛ فضیلت‌ها از طریق مراقبت، حمایت یا پرورش بیان می‌شوند و رذیلت‌ها شامل آسیب‌زدن، بی‌رحمی یا بی‌تفاوتی نسبت به رنج دیگران هستند.


      انصاف در برابر تقلب: ارزش‌های اخلاقی مرتبط با عدالت، حقوق و عمل متقابل؛ انصاف به معنای برابری و پیروی از قوانین است و تقلب شامل بهره‌کشی، فریب یا دست‌کاری است.


      وفاداری در برابر خیانت: اخلاقیات مرتبط با تعلق گروهی؛ وفاداری به معنای همبستگی، تعهد و دفاع از درون‌گروه است، در حالی‌که خیانت به معنای بی‌وفایی یا ترک گروه خود است.


      اقتدار در برابر نافرمانی: اخلاقیات مرتبط با احترام به سنت‌ها و سلسله‌مراتب مشروع؛ اقتدار شامل احترام یا تبعیت از رهبری یا هنجارها است و نافرمانی شامل شورش، بی‌احترامی یا سرپیچی می‌شود.


      پاکی در برابر انحطاط: اخلاقیات مرتبط با پاکی و آلودگی؛ پاکی با نظافت، حیا یا تعالی اخلاقی مرتبط است و انحطاط شامل ناپاکی، فحشا یا فساد درک‌شده می‌شود.


      تحلیل خود را دقیقاً در قالب زیر ارائه دهید:
      hate_label: اگر متن حاوی گفتار نفرت‌انگیز است YES و در غیر این صورتNO
      moral_value: برجسته‌ترین بنیاد اخلاقی را از میان مراقبت، آسیب، انصاف، تقلب، وفاداری، خیانت، اقتدار، نافرمانی، پاکی، انحطاط بنویسید. اگر هیچ بنیان اخلاقی واضحی وجود ندارد، بنویسید None
      explanation: توضیح مختصر و مبتنی بر شواهد ارائه دهید؛ مشخصاً کلمات یا عباراتی را که باعث این طبقه‌بندی اخلاقی شده‌اند ذکر کنید. اگر وجود ندارد، بنویسید None
      فقط و فقط خروجی خواسته‌شده را ارائه دهید و هیچ متن یا توضیح اضافی ننویسید."""

      client = OpenAI(
      api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()

      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

    def mftc_few_with_zero_shot(text, model_name='gpt-4o-mini'):
      prompt= f"""وظیفهٔ شما تحلیل متن "{text}" از نظر وجود گفتار نفرت‌انگیز و شناسایی ابعاد ارزشیِ اخلاقیِ زیربنایی آن است.
      چارچوب نظریهٔ بنیان‌های اخلاقی به دغدغه‌های اصلی اخلاقی و روان‌شناختی اشاره دارد که به‌صورت جفت‌های مثبت در برابر منفی ظاهر می‌شوند:
      مراقبت در برابر آسیب: شامل نگرانی برای رفاه دیگران است؛ فضیلت‌ها از طریق مراقبت، محافظت یا پرورش بیان می‌شوند و رذیلت‌ها شامل آسیب‌زدن، بی‌رحمی یا بی‌تفاوتی نسبت به رنج دیگران هستند.


      انصاف در برابر تقلب: ارزش‌های اخلاقی مرتبط با عدالت، حقوق و عمل متقابل؛ انصاف به معنای برابری، تبعیت از قوانین و عدالت است و تقلب نشان‌دهندهٔ سوءاستفاده، فریب یا دست‌کاری است.


      وفاداری در برابر خیانت: اخلاقیات مرتبط با تعلق به گروه؛ وفاداری به معنای همبستگی، وفاداری به گروه و دفاع از آن است، در حالی که خیانت بیانگر بی‌وفایی یا ترک گروه خود است.


      اقتدار در برابر نافرمانی: اخلاقیات مربوط به احترام به سنت‌ها و سلسله‌مراتب مشروع؛ اقتدار به معنای احترام یا تبعیت از رهبری یا هنجارهاست و نافرمانی شامل شورش، بی‌احترامی یا سرپیچی است.


      پاکی در برابر انحطاط: اخلاقیات مرتبط با پاکی و آلودگی؛ پاکی با نظافت، حیا و تعالی اخلاقی مرتبط است، در حالی که انحطاط شامل ناپاکی، زشتی یا فساد ادراک‌شده است.

      مثال ها:‌
      text:  RT @alikarimi_ak8: کرجِ من \nكرجِ #با_شرف \nكرجِ هميشه قهرمان\n\n#مهسا_امینی \n#حدیث_نجفی \n#وطنم_ایران
      hate_label: No
      moral _value: وفاداری
      explanation: كرجِ #با_شرف

      text: RT @DonMorteza3: خیلی دوست دارم بدونم همسر اون یارو احمقه که میگفت : « اگه پاش بیفته سر ناموسمونم میبریم » دیشب خونه راهش داده یا نه؟! و اگ…
      hate_label: No
      moral_value: آسیب
      Explanation: اگه پاش بیفته سر ناموسمونم میبریم

      text: @IranIntl صحنه سازی اون شبی بود که این ولداازنا نطفه اش بسته شد
      hate_label: No
      moral_value: انحطاط
      explanantion:این ولداازنا نطفه اش بسته ش

      text:  RT @Nazdooneh76: 🔴این بیشرف سکینه حبیبی مدیر مدرسه ولی عصر وحیدیه شهریار هست. روز پنجشنبه دختران وحیدیه درخیابان شروع به اعتراض کردند. شنب…
      hate_label: No
      moral _value: خیانت
      explanation: این بیشرف

      بر اساس مثال‌های بالا، پاسخ خود را دقیقاً در قالب زیر و بدون هیچ متن، توضیح یا توجیه اضافی ارائه دهید:
      hate_label: اگر متن حاوی گفتار نفرت‌انگیز است، بنویسید YES، در غیر این صورت NO
      moral_value: برجسته‌ترین ارزش اخلاقی را از میان مراقبت، آسیب، انصاف، تقلب، وفاداری، خیانت، اقتدار، نافرمانی، پاکی، انحطاط بنویسید. اگر ارزش اخلاقی مشخصی وجود ندارد، بنویسید None
      explanation: توضیحی کوتاه و مبتنی بر شواهد ارائه دهید که به‌طور مشخص به کلمات یا عباراتی که باعث این طبقه‌بندی شده‌اند اشاره کند. اگر وجود ندارد، بنویسید None
      """
      client = OpenAI(
      api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()

      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

    def mftc_cot(text, model_name='gpt-4o-mini'):
      prompt = f"""
      متن "{text}" زیر را برای گفتار نفرت‌انگیز تحلیل کرده و ابعاد ارزش‌های اخلاقی زیربنایی آن را شناسایی کنید:
      از مراحل زیر پیروی کنید:
      مرحله ۱: تعیین کنید که آیا متن حاوی گفتار نفرت‌انگیز هست یا نه. اگر گفتار نفرت‌انگیز وجود دارد، بنویسید YES، در غیر این صورت NO.
      مرحله ۲: چارچوب نظریهٔ بنیان‌های اخلاقی به دغدغه‌های اصلی اخلاقی و روان‌شناختی اشاره دارد که به‌صورت جفت‌های مثبت در برابر منفی بیان می‌شوند:
      مراقبت در برابر آسیب: شامل نگرانی برای رفاه دیگران است؛ فضیلت‌ها از طریق مراقبت، محافظت یا پرورش بیان می‌شوند و رذیلت‌ها شامل آسیب، بی‌رحمی یا بی‌تفاوتی نسبت به رنج دیگران هستند.


      انصاف در برابر تقلب: اخلاق مرتبط با عدالت، حقوق و عمل متقابل؛ انصاف به معنای برابری، قانون‌مداری و تقلب به معنای سوءاستفاده، فریب یا دست‌کاری است.


      وفاداری در برابر خیانت: اخلاق گروه‌محور؛ وفاداری به معنای همبستگی، تعهد و دفاع از گروه، و خیانت به معنای بی‌وفایی یا ترک گروه خود است.


      اقتدار در برابر نافرمانی: اخلاق مرتبط با احترام به سنت‌ها و سلسله‌مراتب مشروع؛ اقتدار به معنای احترام یا تبعیت از رهبری یا هنجارهاست و نافرمانی شامل شورش، بی‌احترامی یا سرپیچی می‌شود.


      پاکی در برابر انحطاط: اخلاق مرتبط با پاکی و آلودگی؛ پاکی با نظافت، حیا یا تعالی اخلاقی مرتبط است و انحطاط شامل ناپاکی، زشتی یا فساد تلقی‌شده می‌باشد.


      مرحله ۳: یکی از ارزش‌های اخلاقی غالب را از میان گزینه‌های زیر انتخاب کنید:
      مراقبت، آسیب، انصاف، تقلب، وفاداری، خیانت، اقتدار، نافرمانی، پاکی، انحطاط
      اگر ارزش اخلاقی مشخصی وجود ندارد، بنویسید None.
      مرحله ۴: توجیهی کوتاه و مبتنی بر شواهد برای انتخاب خود در مرحله ۳ ارائه دهید. مشخصاً کلماتی را که باعث این طبقه‌بندی شدند برجسته کنید.
      پاسخ خود را فقط در قالب زیر ارائه دهید:
      hate_label: YES or NO
      moral_value: ارزش اخلاقی انتخاب‌شده از مرحله ۳. اگر هیچ ارزشی روشن نیست، بنویسید None

      explanation:  noneتوجیه کوتاه از مرحله ۴. اگر وجود ندارد، بنویسید  """

      client = OpenAI(
      api_key = 'gpt_api_key'
          )
      input = [{"role": "system", "content": "You are a social psychologist specializing in moral psychology and hate speech classification"},
              {"role": "user","content": prompt,}]
      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.1
      )
      output = chat_completion.choices[0].message.content.strip()

      hate_label, moral_value, explanation = None, None, None

      lines = output.split('\n')

      for line in lines:
        if line.startswith('hate_label:'):
          hate_label = line.split(':', 1)[1].strip()
        elif line.startswith('moral_value:'):
          moral_value = line.split(':', 1)[1].strip()
        elif line.startswith('explanation:'):
          explanation = line.split(':', 1)[1].strip()

      if not hate_label or not moral_value or not explanation:
        return None
      else:
        return {
            'hate_label' : hate_label,
            'moral_value': moral_value,
            'explanation': explanation
        }

dataset_persia = pd.read_excel('.../MFTCXPLAIN_EMNLP_2025/MFTCxplain-persian.xlsx')

dataset_persia.set_index('id', inplace=True) #reindex to the original 'id' index
print(f"The shape of the entire dataset: {dataset_persia.shape}")
#removing these id because they were used as 4 shot examples
rows_to_remove = [9, 17, 21, 29]
testing = dataset_persia.drop(index=rows_to_remove)
print(f"\nThe shape of the entire dataset minus 4 rows:{testing.shape}")

confirming = dataset_persia.iloc[[9, 17, 21, 29]]
print('\nJust confirming the examples from the few show we will be excluding:\n ')
confirming

The shape of the entire dataset: (608, 9)

The shape of the entire dataset minus 4 rows:(604, 9)

Just confirming the examples from the few show we will be excluding:
 


,tweets,mft_sentiment_first,rationales_first,mft_sentiment_second,rationales_second,mft_sentiment_third,rationales_third,platform,hate_speech
id,,,,,,,,,
10,RT @alikarimi_ak8: کرجِ من \nكرجِ #با_شرف \nكرجِ هميشه قهرمان\n\n#مهسا_امینی \n#حدیث_نجفی \n#وطنم_ایران,LP - Loyaty,كرجِ #با_شرف,NaN,NaN,NaN,NaN,Twitter,0
18,RT @DonMorteza3: خیلی دوست دارم بدونم همسر اون یارو احمقه که میگفت : « اگه پاش بیفته سر ناموسمونم میبریم » دیشب خونه راهش داده یا نه؟! و اگ…,HN - Harm,اگه پاش بیفته سر ناموسمونم میبریم,NaN,NaN,NaN,NaN,Twitter,1
22,@IranIntl صحنه سازی اون شبی بود که این ولداازنا نطفه اش بسته شد,PN - Degradation,این ولداازنا نطفه اش بسته ش,NaN,NaN,NaN,NaN,Twitter,1
30,RT @Nazdooneh76: 🔴این بیشرف سکینه حبیبی مدیر مدرسه ولی عصر وحیدیه شهریار هست. روز پنجشنبه دختران وحیدیه درخیابان شروع به اعتراض کردند. شنب…,LN - Betrayal,این بیشرف,AP - Authority,دختران وحیدیه درخیابان شروع به اعتراض کردند,NaN,NaN,Twitter,0


In [ ]:
gpt_prompts_farsi = gpt_instruct_farsi()
#few
explain_few = dataset_persia.drop(index=rows_to_remove)
explain_few[['hate_label', 'moral_value', 'explanation']] = explain_few['tweets'].progress_apply(lambda x: pd.Series(gpt_prompts_farsi.mftc_few_with_zero_shot(x)))
file_name = f'gpt_persia_mftc_explain_4_shot_with_zero_shot_style_prompt.csv'
file_path = os.path.join(output_path , file_name)
explain_few.to_csv(file_path, index=False)

#zero
explain_zero = dataset_persia.drop(index=rows_to_remove)
explain_zero[['hate_label', 'moral_value', 'explanation']] = explain_zero['tweets'].progress_apply(lambda x: pd.Series(gpt_prompts_farsi.mftc_zero_shot(x)))
file_name = f'gpt_persia_mftc_explain_zero.csv'
file_path = os.path.join(output_path , file_name)
explain_zero.to_csv(file_path, index=False)

#cot
explain_cot = dataset_persia.drop(index=rows_to_remove)
explain_cot[['hate_label', 'moral_value', 'explanation']] = explain_cot['tweets'].progress_apply(lambda x: pd.Series(gpt_prompts_farsi.mftc_cot(x)))
file_name = f'rerun_gpt_persia_mftc_explain_cot.csv'
file_path = os.path.join(output_path , file_name)
explain_cot.to_csv(file_path, index=False)